In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from skimage import color
from skimage import io
import matplotlib.cm as cm
from scipy.misc import imresize
from scipy.ndimage.filters import gaussian_filter
import cv2

C:\Anaconda-python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#mnist 만으로 학습시킨 모델
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../MNIST_datxa/", one_hot=True)

Extracting ../MNIST_datxa/train-images-idx3-ubyte.gz
Extracting ../MNIST_datxa/train-labels-idx1-ubyte.gz
Extracting ../MNIST_datxa/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_datxa/t10k-labels-idx1-ubyte.gz


In [3]:
hnd_train = glob('../../Datas/letter/Hnd/Digit/train/*/*')
hnd_test = glob('../../Datas/letter/Hnd/Digit/test/*/*')
real_train = glob('../../Datas/letter/EnglishImg/English/Img/GoodImg/English_image/Digit/train/*/*')
real_test = glob('../../Datas/letter/EnglishImg/English/Img/GoodImg/English_image/Digit/test/*/*')

In [5]:
def get_label_from_path(path):
    return (int(path.split('\\')[-2]))

def read_image(path):
    img = cv2.imread(path) # read the image from the path
    img = cv2.GaussianBlur(img, (5, 5), 0)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # make gray
    gray_mean = np.mean(gray)
    ret,thresh = cv2.threshold(gray,gray_mean,255,cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU )  #make binary
    kernel = np.ones((1,1), np.uint8) 
    img_dilation = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel) #to get more clear image
    im2,ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #find contours
    x, y, w, h = cv2.boundingRect(ctrs[0]) # make rectangle following out line
    if h > w:
        x -= (h - w)//2
        w += (h - w)
    elif h < w:
        y -= (w - h)//2
        h += (w - h)
    roi = img_dilation[y:y+h, x:x+w]
    roi_resized = imresize(roi, (28, 28)).reshape(784)
    roi_resized = np.array(roi_resized, dtype= np.float32)
    for i in range(len(roi_resized)):
        roi_resized[i] /= 255
    return roi_resized


def read_image_not_roi(path):
    img = cv2.imread(path) # read the image from the path
    img = cv2.GaussianBlur(img, (5, 5), 0)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # make gray
    gray_mean = np.mean(gray)
    ret,thresh = cv2.threshold(gray,gray_mean,255,cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU )  #make binary
    kernel = np.ones((1,1), np.uint8) 
    img_dilation = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel) #to get more clear image
    if(num_dict[255] >= num_dict[0]):
        ret,img_dilation = cv2.threshold(img_dilation,gray_mean,255,cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU )
    image = imresize(img_dilation, (28, 28)).reshape(784)
    image = np.array(image, dtype= np.float32)
    for i in range(len(image)):
        image[i] /= 255
    return image

path = hnd_train[2]

path, get_label_from_path(path)
class_name = get_label_from_path(path)

label_name_list_train = []

for path in hnd_train:
    label_name_list_train.append(get_label_from_path(path))

    
unique_label_names = np.unique(label_name_list_train)

def onehot_encode_label(path):
    onehot_label = unique_label_names == get_label_from_path(path)
    onehot_label = onehot_label.astype(np.uint8)
    return onehot_label

In [6]:
onehot_encode_label(path)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [7]:
def make_dataset(image_shape, label_shape, path_images):
    initial_image_array = np.zeros([len(path_images),image_shape])
    initial_label_array = np.zeros([len(path_images),label_shape])
    for n, path in enumerate(path_images):
        image = read_image(path)
        onehot_label = onehot_encode_label(path)
        initial_image_array[n, :] = image
        initial_label_array[n, :] = onehot_label
    return initial_image_array, initial_label_array

def make_dataset_real(image_shape, label_shape, path_images):
    initial_image_array = np.zeros([len(path_images),image_shape])
    initial_label_array = np.zeros([len(path_images),label_shape])
    for n, path in enumerate(path_images):
        image = read_image_not_roi(path)
        onehot_label = onehot_encode_label(path)
        initial_image_array[n, :] = image
        initial_label_array[n, :] = onehot_label
    return initial_image_array, initial_label_array

In [8]:
data_train_images_hnd, data_train_labels_hnd = make_dataset(784, 10, hnd_train)
data_test_images_hnd, data_test_labels_hnd = make_dataset(784, 10, hnd_test)

C:\Anaconda-python\lib\site-packages\ipykernel_launcher.py:91: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [9]:
data_test_images_real, data_test_labels_real = make_dataset_real(784, 10, real_test)
data_train_images_real, data_train_labels_real =make_dataset_real(784, 10, real_train)

C:\Anaconda-python\lib\site-packages\ipykernel_launcher.py:106: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [10]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100


# The model trained with hand data

# tensorboard.ver

In [13]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.int8, [None, 10])
keep_prob = tf.placeholder(tf.float32)

TB_SUMMARY_DIR = './tb/mnist'

with tf.variable_scope('layer1') as scope:
    W1 = tf.get_variable("W", shape=[784, 512],
                         initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([512]))
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

    tf.summary.histogram("X", X)
    tf.summary.histogram("weights", W1)
    tf.summary.histogram("bias", b1)
    tf.summary.histogram("layer", L1)

with tf.variable_scope('layer2') as scope:
    W2 = tf.get_variable("W", shape=[512, 256],
                         initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([256]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

    tf.summary.histogram("weights", W2)
    tf.summary.histogram("bias", b2)
    tf.summary.histogram("layer", L2)

with tf.variable_scope('layer3') as scope:
    W3 = tf.get_variable("W", shape=[256, 128],
                         initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([128]))
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

    tf.summary.histogram("weights", W3)
    tf.summary.histogram("bias", b3)
    tf.summary.histogram("layer", L3)

with tf.variable_scope('layer4') as scope:
    W4 = tf.get_variable("W", shape=[128, 64],
                         initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([64]))
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

    tf.summary.histogram("weights", W4)
    tf.summary.histogram("bias", b4)
    tf.summary.histogram("layer", L4)

with tf.variable_scope('layer5') as scope:
    W5 = tf.get_variable("W", shape=[64, 10],
                         initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([10]))
    hypothesis = tf.matmul(L4, W5) + b5

    tf.summary.histogram("weights", W5)
    tf.summary.histogram("bias", b5)
    tf.summary.histogram("hypothesis", hypothesis)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

tf.summary.scalar("loss", cost)
summary = tf.summary.merge_all()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter(TB_SUMMARY_DIR)
writer.add_graph(sess.graph)
global_step = 0

print('hand write data training start')
for i in range(len(hnd_train)):
    feed_dict = {X: data_train_images_hnd, Y: data_train_labels_hnd, keep_prob:0.7}
    s, _ = sess.run([summary, optimizer], feed_dict=feed_dict)
    writer.add_summary(s, global_step=global_step)
    global_step += 1
    if i % 100 == 0:
        print('step: {0}, cost: {1}'.format(i, cost))
print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_hnd, Y: data_test_labels_hnd, keep_prob: 1}))

hand write data training start
step: 0, cost: Tensor("Mean:0", shape=(), dtype=float32)
step: 100, cost: Tensor("Mean:0", shape=(), dtype=float32)
step: 200, cost: Tensor("Mean:0", shape=(), dtype=float32)
step: 300, cost: Tensor("Mean:0", shape=(), dtype=float32)
step: 400, cost: Tensor("Mean:0", shape=(), dtype=float32)
Learning Finished!
Accuracy: 0.9727273


In [14]:
#mnist data's accuracy and  real image data's accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))


Accuracy: 0.2131


In [15]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_real, Y: data_test_labels_real, keep_prob: 1}))


Accuracy: 0.16891892
